In [ ]:
%pip install --quiet --upgrade diffusers transformers accelerate mediapy scipy ftfy spacy

In [ ]:
# The xformers package is mandatory to be able to create several 768x768 images.
github_url = "https://github.com/brian6091/xformers-wheels"
xformer_id = "0.0.15.dev0+4c06c79"
xformers_wheels = f"xformers-{xformer_id}.d20221205-cp38-cp38-linux_x86_64.whl"

# Install xformers using pre-compiled Python wheels
%pip install -q {github_url}/releases/download/{xformer_id}/{xformers_wheels}

In [ ]:
# model_id = "stabilityai/stable-diffusion-2-1-base"
# model_id = "stabilityai/stable-diffusion-2-1"
model_id = "dreamlike-art/dreamlike-photoreal-2.0"

In [ ]:
from diffusers import PNDMScheduler, DDIMScheduler, LMSDiscreteScheduler, EulerDiscreteScheduler, DPMSolverMultistepScheduler

scheduler = None
# scheduler = PNDMScheduler.from_pretrained(model_id, subfolder="scheduler")
# scheduler = DDIMScheduler.from_pretrained(model_id, subfolder="scheduler")
# scheduler = LMSDiscreteScheduler.from_pretrained(model_id, subfolder="scheduler")
# scheduler = EulerDiscreteScheduler.from_pretrained(model_id, subfolder="scheduler")
# scheduler = DPMSolverMultistepScheduler.from_pretrained(model_id, subfolder="scheduler")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import mediapy as media
import torch
from diffusers import StableDiffusionPipeline

device = "cuda"

if model_id.startswith("stabilityai/"):
  model_revision = "fp16"
else:
  model_revision = None

if scheduler is None:
  pipe = StableDiffusionPipeline.from_pretrained(
      model_id,
      torch_dtype=torch.float16,
      revision=model_revision,
      )  
else:
  pipe = StableDiffusionPipeline.from_pretrained(
      model_id,
      scheduler=scheduler,
      torch_dtype=torch.float16,
      revision=model_revision,
      )

pipe = pipe.to(device)
pipe.enable_xformers_memory_efficient_attention()

if model_id.endswith('-base'):
  image_length = 512
else:
  image_length = 768


In [ ]:
#Install the Pokedex API to get Pokemon data. I want this for the Pokemon descriptions. These will serve as the prompts.
%pip install git+https://github.com/arnavb/pypokedex.git@master

In [ ]:
import pypokedex
#Specify which Pokemon we want to query using its ID number and from what game to get the description from
DEX = 4
VER = 'yellow'

#Specify which Pokemon we want to query using its ID number
pokemon = pypokedex.get(dex=DEX)
print(pokemon)

#This is the name of the Pokemon we are querying
poke_name = pokemon.name

#This is the PokeDex desciption for the current Pokemon
yellow_description = pokemon.get_descriptions()[VER]

#This is the prompt I'll feed to the AI 
prompt = poke_name + " " + yellow_description
print(prompt)

In [ ]:
remove_safety = False
num_images = 4

if remove_safety:
  negative_prompt = None
else:
  negative_prompt = "nude, naked"

images = pipe(
    prompt,
    height = image_length,
    width = image_length,
    num_inference_steps = 25,
    guidance_scale = 9,
    num_images_per_prompt = num_images,
    negative_prompt = negative_prompt,
    ).images
    
media.show_images(images)
#images[0].save(r'output.jpg')

In [ ]:
import pypokedex

def get_concat_h(im1, im2):
    dst = Image.new('RGB', (im1.width + im2.width, im1.height))
    dst.paste(im1, (0, 0))
    dst.paste(im2, (im1.width, 0))
    return dst

def get_concat_h_multi_blank(im_list):
    _im = im_list.pop(0)
    for im in im_list:
        _im = get_concat_h(_im, im)
    return _im
    
def makePokemonFromPokedex(ver,nPokemon):
  #Loop over nPokemons to get the descritptions and generate images for each
  #poke_id = 1
  for poke_id in range(1, nPokemon+1, 1):
    #print(poke_id)
    #Specify which Pokemon we want to query using its ID number
    pokemon = pypokedex.get(dex=poke_id)
    #print(pokemon)

    #This is the name of the Pokemon we are querying
    poke_name = pokemon.name

    #This is the PokeDex desciption for the current Pokemon
    yellow_description = pokemon.get_descriptions()[ver]

    #This is the prompt I'll feed to the AI 
    prompt = poke_name + " " + yellow_description
    #print(prompt)

    remove_safety = False
    num_images = 4

    if remove_safety:
      negative_prompt = None
    else:
      negative_prompt = "nude, naked"

    images = pipe(
        prompt,
        height = image_length,
        width = image_length,
        num_inference_steps = 25,
        guidance_scale = 9,
        num_images_per_prompt = num_images,
        negative_prompt = negative_prompt,
        ).images
    
    fname = 'poke_' + str(poke_id)
    get_concat_h_multi_blank(images).save(fname + '.jpg')
#get_concat_h_multi_blank(images).save('test2.jpg')


In [ ]:
#get_concat_h(images[0], images[1]).save('grid0.jpg')
#img1 = append_images([images[0], images[1]], direction='horizontal')
#img2 = append_images([image512, image1024], direction='horizontal')
#final = append_images([img1, img2], direction='vertical')
#final.save("Test/test.jpg")

In [ ]:
def makePokemonFromPokedex(ver,nPokemon):
  #Loop over nPokemons to get the descritptions and generate images for each
  #poke_id = 1
  for poke_id in range(1, nPokemon+1, 1):
    #print(poke_id)
    #Specify which Pokemon we want to query using its ID number
    pokemon = pypokedex.get(dex=poke_id)
    #print(pokemon)

    #This is the name of the Pokemon we are querying
    poke_name = pokemon.name

    #This is the PokeDex desciption for the current Pokemon
    yellow_description = pokemon.get_descriptions()[ver]

    #This is the prompt I'll feed to the AI 
    prompt = poke_name + " " + yellow_description
    #print(prompt)

    remove_safety = False
    num_images = 4

    if remove_safety:
      negative_prompt = None
    else:
      negative_prompt = "nude, naked"

    images = pipe(
        prompt,
        height = image_length,
        width = image_length,
        num_inference_steps = 25,
        guidance_scale = 9,
        num_images_per_prompt = num_images,
        negative_prompt = negative_prompt,
        ).images
    
    fname = '/content/AI_Pokemon/poke_' + str(poke_id)
    get_concat_h_multi_blank(images).save(fname + '.jpg')
   
  #media.show_images(images)
  #images[0].save(r'output.jpg')

In [ ]:
!pip install Pillow

In [ ]:
import PIL
from PIL import Image
print(PIL.PILLOW_VERSION)

In [ ]:
makePokemonFromPokedex('yellow',150)

In [ ]:
DEX = 1
VER = 'yellow'

#Specify which Pokemon we want to query using its ID number
pokemon = pypokedex.get(dex=DEX)
print(pokemon)

#This is the name of the Pokemon we are querying
poke_name = pokemon.name

#This is the PokeDex desciption for the current Pokemon
yellow_description = pokemon.get_descriptions()[VER]

#This is the prompt I'll feed to the AI 
prompt = poke_name + " " + yellow_description
print(prompt)

remove_safety = False
num_images = 4

if remove_safety:
  negative_prompt = None
else:
  negative_prompt = "nude, naked"

images = pipe(
    prompt,
    height = image_length,
    width = image_length,
    num_inference_steps = 25,
    guidance_scale = 9,
    num_images_per_prompt = num_images,
    negative_prompt = negative_prompt,
    ).images
    
media.show_images(images)
images[0].save(r'output.jpg')

In [ ]:
from google.colab import files

In [ ]:
!zip -r /content/AI_Pokemon/sample_data.zip /content/AI_Pokemon

In [ ]:
DEX = 1
VER = 'yellow'

#Specify which Pokemon we want to query using its ID number
pokemon = pypokedex.get(dex=DEX)
print(pokemon)

#This is the name of the Pokemon we are querying
poke_name = pokemon.name

#This is the PokeDex desciption for the current Pokemon
yellow_description = pokemon.get_descriptions()[VER]

#This is the prompt I'll feed to the AI 
prompt = poke_name + " " + yellow_description
print(prompt)

remove_safety = False
num_images = 4

if remove_safety:
  negative_prompt = None
else:
  negative_prompt = "nude, naked"

images = pipe(
    prompt,
    height = image_length,
    width = image_length,
    num_inference_steps = 25,
    guidance_scale = 9,
    num_images_per_prompt = num_images,
    negative_prompt = negative_prompt,
    ).images
    
media.show_images(images)
images[0].save(r'output.jpg')

In [ ]:
with open('example.txt', 'w') as f:
  f.write('some content')

files.download('example.txt')